# 1) pkg installation and importing

1) install pkgs

In [ ]:
try:
  # global
  from typing import List, Optional, Type, Union
  import builtins

  import ivy
  import ivy_models
  from ivy_models.resnet.layers import conv1x1, BasicBlock, Bottleneck
  from ivy_models.base import BaseSpec, BaseModel
except:
  ! pip install git+https://github.com/unifyai/ivy.git

  # global
  from typing import List, Optional, Type, Union
  import builtins

  import ivy
  import ivy_models
  from ivy_models.resnet.layers import conv1x1, BasicBlock, Bottleneck
  from ivy_models.base import BaseSpec, BaseModel


In [ ]:
# import ivy_models
import ivy
import torch
ivy.set_backend('torch')
import ivy
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')
import pprint
ivy.set_backend('torch')
# !rm -r /content/models
# !git clone https://github.com/unifyai/models.git
# !cd models && python3 -m pip install -q -e .

!python3 -m pip install torchvision

2) import pkgs

In [ ]:
# None

# 2) Test_Image Prep

1) download and load image

In [ ]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

from PIL import Image
dog_image = Image.open(filename)
# display(dog_image)

2) preprocess image for torch model

In [ ]:
#############################################################################################
# Preprocess image for torch model
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])
# torch_img = Image.open("/kaggle/working/dog.jpg")
torch_img = preprocess(dog_image)
torch_img = torch.unsqueeze(torch_img, 0)

#############################################################################################
# preprocess image for ivy model
def load_and_preprocess_img(
    path,
    new_size,
    crop,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
    data_format="NHWC",
    to_ivy=False,
):
    img = Image.open(path)
    compose = transforms.Compose(
        [
            transforms.Resize(new_size),
            transforms.CenterCrop(crop),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ]
    )
    img = compose(img)
    img = img.unsqueeze(0)
    if data_format == "NHWC":
        img = img.permute((0, 2, 3, 1))
    return ivy.array(img.numpy()) if to_ivy else img.numpy()

# Preprocess ivy image
ivy_img = load_and_preprocess_img(
        filename,
        256,
        224,
        to_ivy=True,)
print(ivy_img.shape)
# ivy_img = torch_img.permute((0, 2, 3, 1)).detach().numpy()

ivy.Shape(1, 224, 224, 3)


# 4.1) both model compilation

1) compiling ivy model

1.1) defining dependency funcs for weight prep

In [ ]:
def _prune_keys(raw, ref, raw_keys_to_prune=[], ref_keys_to_prune=[]):
    pruned_ref = {}
    if raw_keys_to_prune:
        raw = raw.cont_prune_keys(raw_keys_to_prune)
    if ref_keys_to_prune:
        pruned_ref = ref.cont_at_keys(ref_keys_to_prune)
        ref = ref.cont_prune_keys(ref_keys_to_prune)
    return raw, ref, pruned_ref


def _map_weights(raw, ref, custom_mapping=None):
    mapping = {}
    for old_key, new_key in zip(
        raw.cont_sort_by_key().cont_to_iterator_keys(),
        ref.cont_sort_by_key().cont_to_iterator_keys(),
    ):
        new_mapping = new_key
        if custom_mapping is not None:
            new_mapping = custom_mapping(old_key, new_key)
        mapping[old_key] = new_mapping
    return mapping


import pprint
def load_torch_weights(
    url,
    ref_model,
    raw_keys_to_prune=[],
    ref_keys_to_prune=[],
    custom_mapping=None,
    map_location=torch.device("cpu"),
):
    ivy.set_backend("torch")
    weights = torch.hub.load_state_dict_from_url(url, map_location=map_location)
    display(f"downloaded weights are:");pprint.pprint(ivy.asarray(ivy.Container(weights)))
    display("load_torch_weights | done 1/3")

    weights_raw = ivy.to_numpy(ivy.Container(weights))
    # display(f"numpied weights are:");pprint.pprint(ivy.asarray(ivy.Container(weights)))
    display("load_torch_weights | done 2/6")

    weights_raw, weights_ref, pruned_ref = _prune_keys(weights_raw, ref_model.v, raw_keys_to_prune, ref_keys_to_prune)
    # display(f"weights_raw are:");pprint.pprint(ivy.asarray(ivy.Container(weights_raw)))
    # display(f"weights_ref are:");pprint.pprint(ivy.Container(weights_ref))
    # display(f"pruned_ref are:");pprint.pprint(ivy.Container(pruned_ref))
    display("load_torch_weights | done 3/6")

    mapping = _map_weights(weights_raw, weights_ref, custom_mapping=custom_mapping)
    # display(f"mapping are:");pprint.pprint(ivy.Container(mapping))
    display("load_torch_weights | done 4/6")

    ivy.previous_backend()
    w_clean = weights_raw.cont_restructure(mapping, keep_orig=False)
    # display(f"mapping are:");pprint.pprint(ivy.Container(w_clean))
    display("load_torch_weights | done 5/6")

    if ref_keys_to_prune:
        w_clean = ivy.Container.cont_combine(w_clean, pruned_ref)
        display("ref_keys_to_prune is True")
        # display(f"w_clean are:");pprint.pprint(ivy.asarray(ivy.Container(w_clean)))
    display("load_torch_weights | done 6/6")

    ret = ivy.asarray(w_clean)
    display("load_torch_weights | done 7/7")

    return ret

1.2) defining ivy model compilation function

In [ ]:
def _inceptionNet_v3_torch_weights_mapping(old_key, new_key):
    W_KEY = ["conv/weight"]
    new_mapping = new_key
    if any([kc in old_key for kc in W_KEY]):
        new_mapping = {"key_chain": new_key, "pattern": "b c h w -> h w c b"}
    return new_mapping


def ivy_inceptionNet_v3(pretrained=True):
    """InceptionNet-V3 model"""

    model = InceptionV3()
    # display(f"my model weights are:");pprint.pprint(ivy.Container(model.v))
    display("_inceptionNet_v3_torch_weights_mapping | done 1/3")
    if pretrained:
        url = "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth"
        w_clean = load_torch_weights(
            url,
            model,
            raw_keys_to_prune=["num_batches_tracked", "AuxLogits"],
            custom_mapping=_inceptionNet_v3_torch_weights_mapping,
        )
        display("_inceptionNet_v3_torch_weights_mapping | done 2/3")

        model.v = w_clean
        display("_inceptionNet_v3_torch_weights_mapping | done 3/3")
    return model

2) compiling ivy model

In [ ]:
!echo -n gDNLm8VEO7QleBN5GdTcfMa7OHWBOWcyOiHWr2KI654= > .ivy/key.pem

# predicting image using IVY googlenet
ivy.set_backend('torch')
ivy_inception_v31 = ivy_inceptionNet_v3(pretrained=True)

# random_test_tensor = ivy.random_normal(shape=(1, 224, 224, 3))
# ivy_inception_v3.compile(args=(random_test_tensor,))

/bin/bash: line 1: .ivy/key.pem: No such file or directory


NameError: ignored

2) compiling torch model

In [ ]:
# predicting torch_img using torch_googlenet
from torchvision.models import inception_v3 as torch_inception_v3_model
torch_inception_v3 = torch_inception_v3_model(pretrained=True)
torch_inception_v3.eval()

# 4.2) forward pass inference / predicting the class of dog image for accuracy check

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
torch_output = torch.softmax(torch_inception_v3(torch_img), dim=1)
torch_classes = torch.argsort(torch_output[0], descending=True)[:3]
torch_logits = torch.take(torch_output[0], torch_classes)

print("Indices of the top 3 classes are:", torch_classes)
print("Logits of the top 3 classes are:", torch_logits)
print("Categories of the top 3 classes are:", [categories[i] for i in torch_classes])

Indices of the top 3 classes are: tensor([258, 270, 279])
Logits of the top 3 classes are: tensor([9.9990e-01, 8.3909e-05, 1.1693e-05], grad_fn=<TakeBackward0>)
Categories of the top 3 classes are: ['Samoyed', 'white wolf', 'Arctic fox']


In [ ]:
# output = ivy.softmax(ivy_inception_v3(tuple(ivy_img))[0])  # pass the image to the model
output = ivy.softmax(ivy_inception_v31(ivy_img))  # pass the image to the model
classes = ivy.argsort(output[0], descending=True)[:3]  # get the top 3 classes
logits = ivy.gather(output[0], classes)  # get the logits

print("Indices of the top 3 classes are:", classes)
print("Logits of the top 3 classes are:", logits)
print("Categories of the top 3 classes are:", [categories[i] for i in classes.to_list()])

NameError: ignored

In [ ]:
assert output.shape == tuple([1, 1000])

In [ ]:
assert np.array_equal(torch_classes, classes)

In [ ]:
assert torch.allclose(torch_logits, logits, rtol=1)

# 3) Building the Incpetion V3 Model Architecture

In [1]:
try:
  import ivy
  from typing import *

  print("cool!")

except:
  print('lul')

  !pip install git+https://github.com/unifyai/ivy.git

  import ivy
  from typing import *


cool!


In [ ]:
class BasicConv2d(ivy.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super().__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding, with_bias=False)
        self.bn = ivy.BatchNorm2D(self.out_channels, eps=0.001)

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = ivy.relu(x)
        return x

def test_BasicConv2d():
    # N x 768 x 5 x 5
    random_test_tensor = ivy.random_normal(shape=(1, 5, 5, 768))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    block = BasicConv2d(768, 128, [1,1])
    block(random_test_tensor)
    # N x 128 x 5 x 5
    display("Test Successfull!")

test_BasicConv2d()

'random_test_tensor shape is: ivy.Shape(1, 5, 5, 768)'

'Test Successfull!'

In [ ]:
class InceptionAux(ivy.Module):
    def __init__(self, in_channels: int, num_classes: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels
        self.num_classes = num_classes

    def _build(self, *args, **kwargs):
        self.conv0 = self.conv_block(self.in_channels, 128, kernel_size=[1,1])
        self.conv1 = self.conv_block(128, 768, kernel_size=[5,5])
        self.conv1.stddev = 0.01  # type: ignore[assignment]
        self.fc = ivy.Linear(768, self.num_classes)
        self.fc.stddev = 0.001  # type: ignore[assignment]

    def _forward(self, x):
        # N x 768 x 17 x 17
        display(f"InceptionAux | input shape is:{x.shape}")
        x = ivy.avg_pool2d(x, [5,5], 3, 'valid', data_format='NHWC')
        display(f"InceptionAux | done 1/8, output shape is:{x.shape}")

        # N x 768 x 5 x 5
        x = self.conv0(x)
        display("InceptionAux | done 2/8")

        # N x 128 x 5 x 5
        x = self.conv1(x)
        display("InceptionAux | done 3/8")

        # N x 768 x 1 x 1
        # Adaptive average pooling
        display(f"InceptionAux | input shape to adaptive_avg_pool2d is:{x.shape}")
        x = ivy.permute_dims(x, (0, 3, 1, 2))
        display(f"InceptionAux | permuted input shape to adaptive_avg_pool2d is:{x.shape}")
        x = ivy.adaptive_avg_pool2d(x, (1, 1))
        display(f"InceptionAux | output shape from adaptive_avg_pool2d is:{x.shape}")
        x = ivy.permute_dims(x, (0, 2, 3, 1))
        display(f"InceptionAux | permuted output shape from adaptive_avg_pool2d is:{x.shape}")
        display("InceptionAux | done 4/8")

        # N x 768 x 1 x 1
        x = ivy.flatten(x, start_dim=1)
        display("InceptionAux | done 5/8")

        # N x 768
        x = self.fc(x)
        display("InceptionAux | done 8/8")
        # N x 1000
        return x


def test_InceptionAux():
    random_test_tensor = ivy.random_normal(shape=(1, 17, 17, 768))
    display(f"InceptionAux | random_test_tensor shape is: {random_test_tensor.shape}")

    block = InceptionAux(768, 1000)
    block(random_test_tensor)
    display("InceptionAux | Test Successfull!")

test_InceptionAux()

'InceptionAux | random_test_tensor shape is: ivy.Shape(1, 17, 17, 768)'

'InceptionAux | input shape is:ivy.Shape(1, 17, 17, 768)'

'InceptionAux | done 1/8, output shape is:ivy.Shape(1, 5, 5, 768)'

'InceptionAux | done 2/8'

'InceptionAux | done 3/8'

...

'InceptionAux | permuted output shape from adaptive_avg_pool2d is:ivy.Shape(1, 1, 1, 768)'

'InceptionAux | done 4/8'

'InceptionAux | done 5/8'

'InceptionAux | done 8/8'

'InceptionAux | Test Successfull!'

In [ ]:
class InceptionE(ivy.Module):
    def __init__(self, in_channels: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 320, kernel_size=[1,1])

        self.branch3x3_1 = self.conv_block(self.in_channels, 384, kernel_size=[1,1])
        self.branch3x3_2a = self.conv_block(384, 384, kernel_size=[1, 3], padding=[[0,0], [1,1]])
        self.branch3x3_2b = self.conv_block(384, 384, kernel_size=[3, 1], padding=[[1,1], [0,0]])

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 448, kernel_size=[1,1])
        self.branch3x3dbl_2 = self.conv_block(448, 384, kernel_size=[3,3], padding=1)
        self.branch3x3dbl_3a = self.conv_block(384, 384, kernel_size=[1, 3], padding=[[0,0], [1,1]])
        self.branch3x3dbl_3b = self.conv_block(384, 384, kernel_size=[3, 1], padding=[[1,1], [0,0]])

        self.branch_pool = self.conv_block(self.in_channels, 192, kernel_size=[1,1])

#         self.avg_pool = ivy.AvgPool2D([3,3], (1,1), [[1,1],[1,1]])

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        branch1x1 = self.branch1x1(x)
        display(f"1/20, branch1x1 output shape is:{branch1x1.shape}")

        branch3x3 = self.branch3x3_1(x)
        display(f"2/20, branch3x3 output shape is:{branch3x3.shape}")
        branch3x3 = ivy.concat([self.branch3x3_2a(branch3x3), self.branch3x3_2b(branch3x3),], axis=3)
        display(f"3/20, branch3x3 output shape is:{branch3x3.shape}")

        branch3x3dbl = self.branch3x3dbl_1(x)
        display(f"4/20, branch3x3dbl output shape is:{branch3x3dbl.shape}")
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        display(f"5/20, branch3x3dbl output shape is:{branch3x3dbl.shape}")
        branch3x3dbl = ivy.concat([self.branch3x3dbl_3a(branch3x3dbl), self.branch3x3dbl_3b(branch3x3dbl),], axis=3)
        display(f"6/20, branch3x3dbl output shape is:{branch3x3dbl.shape}")

        #XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
        import pprint
        # pprint.pprint(x)

        #--------------------------------
        import json
        import numpy as np

        # Create a NumPy array
        arr = np.array(x)

        # Convert the NumPy array to a Python list
        arr_list = arr.tolist()

        # Define the path and filename for the JSON file
        file_path = '/content/file.json'

        # Save the NumPy array as JSON
        with open(file_path, 'w') as json_file:
            json.dump(arr_list, json_file)
        #XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


        branch_pool = ivy.avg_pool2d(x, [3,3], (1,1), [(1,1),(1,1)])
#         branch_pool = self.avg_pool(x)
        branch_pool = self.branch_pool(branch_pool)
        display(f"7/20, branch_pool output shape is:{branch_pool.shape}")

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        display(f"20/20")
        return outputs


def test_InceptionE():
    random_test_tensor = ivy.random_normal(shape=(1, 8, 8, 1280))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    # N x 1280 x 8 x 8
    block = InceptionE(1280)
    block(random_test_tensor)
    # N x 2048 x 8 x 8
    display("Test Successfull!")

test_InceptionE()

'random_test_tensor shape is: ivy.Shape(1, 8, 8, 1280)'

'input shape is:ivy.Shape(1, 8, 8, 1280)'

'1/20, branch1x1 output shape is:ivy.Shape(1, 8, 8, 320)'

'2/20, branch3x3 output shape is:ivy.Shape(1, 8, 8, 384)'

'3/20, branch3x3 output shape is:ivy.Shape(1, 8, 8, 768)'

...

'5/20, branch3x3dbl output shape is:ivy.Shape(1, 8, 8, 384)'

'6/20, branch3x3dbl output shape is:ivy.Shape(1, 8, 8, 768)'

'7/20, branch_pool output shape is:ivy.Shape(1, 8, 8, 192)'

'20/20'

'Test Successfull!'

In [ ]:
# note:
# kernal_size in [1,3] list foramt
# and padding in list[list, list] foramt ex: [[1,1],[3,3]] if both dims are unequal, else just single ex: list [3,3]
class InceptionD(ivy.Module):
    def __init__(self, in_channels: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels

    def _build(self, *args, **kwargs):
        self.branch3x3_1 = self.conv_block(self.in_channels, 192, kernel_size=[1,1])
        display(f"layer 1/6 built")
        self.branch3x3_2 = self.conv_block(192, 320, kernel_size=[3,3], stride=2)
        display(f"layer 2/6 built")

        self.branch7x7x3_1 = self.conv_block(self.in_channels, 192, kernel_size=[1,1])
        display(f"layer 3/6 built")
        self.branch7x7x3_2 = self.conv_block(192, 192, kernel_size=[1,7], padding=[[0,0], [3,3]])
        display(f"layer 4/6 built")
        self.branch7x7x3_3 = self.conv_block(192, 192, kernel_size=[7,1], padding=[[3,3], [0,0]])
        display(f"layer 5/6 built")
        self.branch7x7x3_4 = self.conv_block(192, 192, kernel_size=[3,3], stride=2)
        display(f"layer 6/6 built")

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        branch3x3 = self.branch3x3_1(x)
        display(f"one 1/20, output shape is:{branch3x3.shape}")
        branch3x3 = self.branch3x3_2(branch3x3)
        display(f"one 2/20, output shape is:{branch3x3.shape}")

        branch7x7x3 = self.branch7x7x3_1(x)
        display(f"one 3/20, output shape is:{branch7x7x3.shape}")
        branch7x7x3 = self.branch7x7x3_2(branch7x7x3)
        display(f"one 4/20, output shape is:{branch7x7x3.shape}")
        branch7x7x3 = self.branch7x7x3_3(branch7x7x3)
        display(f"one 5/20, output shape is:{branch7x7x3.shape}")
        branch7x7x3 = self.branch7x7x3_4(branch7x7x3)
        display(f"one 6/20, output shape is:{branch7x7x3.shape}")

        branch_pool = ivy.max_pool2d(x, [3,3], 2, 0)
        display(f"one 7/20, output shape is:{branch_pool.shape}")

        outputs = [branch3x3, branch7x7x3, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        display(f"one 20/20")

        return outputs


def test_InceptionD():
    random_test_tensor = ivy.random_normal(shape=(1, 17, 17, 768))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    # N x 768 x 17 x 17
    block = InceptionD(768)
    block(random_test_tensor)
    # N x 1280 x 8 x 8
    display("Test Successfull!")

test_InceptionD()

'random_test_tensor shape is: ivy.Shape(1, 17, 17, 768)'

'layer 1/6 built'

'layer 2/6 built'

'layer 3/6 built'

'layer 4/6 built'

...

'one 5/20, output shape is:ivy.Shape(1, 17, 17, 192)'

'one 6/20, output shape is:ivy.Shape(1, 8, 8, 192)'

'one 7/20, output shape is:ivy.Shape(1, 8, 8, 768)'

'one 20/20'

'Test Successfull!'

In [ ]:
# note:
# kernal_size in [1,3] list foramt
# and padding in list[list, list] foramt ex: [[1,1],[3,3]] if both dims are unequal, else just single ex: list [3,3]
class InceptionC(ivy.Module):
    def __init__(self, in_channels: int, channels_7x7: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels
        self.channels_7x7 = channels_7x7

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 192, kernel_size=[1,1])
        display(f"layer 1/10 built")

        c7 = self.channels_7x7
        self.branch7x7_1 = self.conv_block(self.in_channels, c7, kernel_size=[1,1])
        display(f"layer 2/10 built")
        self.branch7x7_2 = self.conv_block(c7, c7, kernel_size=[1, 7], padding=[[0,0],[3,3]])
        display(f"layer 3/10 built")
        self.branch7x7_3 = self.conv_block(c7, 192, kernel_size=[7, 1], padding=[[3,3],[0,0]])
        display(f"layer 4/10 built")
        self.branch7x7dbl_1 = self.conv_block(self.in_channels, c7, kernel_size=[1,1])
        display(f"layer 5/10 built")
        self.branch7x7dbl_2 = self.conv_block(c7, c7, kernel_size=[7, 1], padding=[[3,3],[0,0]])
        display(f"layer 6/10 built")
        self.branch7x7dbl_3 = self.conv_block(c7, c7, kernel_size=[1, 7], padding=[[0,0],[3,3]])
        display(f"layer 7/10 built")
        self.branch7x7dbl_4 = self.conv_block(c7, c7, kernel_size=[7, 1], padding=[[3,3],[0,0]])
        display(f"layer 8/10 built")
        self.branch7x7dbl_5 = self.conv_block(c7, 192, kernel_size=[1, 7], padding=[[0,0],[3,3]])
        display(f"layer 9/10 built")

        self.branch_pool = self.conv_block(self.in_channels, 192, kernel_size=[1,1])
        display(f"layer 10/10 built")

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        branch1x1 = self.branch1x1(x)
        display(f"one 1/20, output shape is:{branch1x1.shape}")

        branch7x7 = self.branch7x7_1(x)
        display(f"one 2/20, output shape is:{branch7x7.shape}")
        branch7x7 = self.branch7x7_2(branch7x7)
        display(f"one 3/20, output shape is:{branch7x7.shape}")
        branch7x7 = self.branch7x7_3(branch7x7)
        display(f"one 4/20, output shape is:{branch7x7.shape}")

        branch7x7dbl = self.branch7x7dbl_1(x)
        display(f"one 5/20, output shape is:{branch7x7dbl.shape}")
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        display(f"one 6/20, output shape is:{branch7x7dbl.shape}")
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        display(f"one 7/20, output shape is:{branch7x7dbl.shape}")
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        display(f"one 8/20, output shape is:{branch7x7dbl.shape}")
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)
        display(f"one 9/20, output shape is:{branch7x7dbl.shape}")

        branch_pool = ivy.avg_pool2d(x, [3,3], [1,1], [[1,1],[1,1]])
        display(f"one 10/20, output shape is:{branch_pool.shape}")
        branch_pool = self.branch_pool(branch_pool)
        display(f"one 11/20, output shape is:{branch_pool.shape}")

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        display(f"one 20/20")

        return outputs


def test_InceptionC():
    random_test_tensor = ivy.random_normal(shape=(1, 17, 17, 768))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    # N x 768 x 17 x 17
    block = InceptionC(768, channels_7x7=128)
    block(random_test_tensor)
    # N x 768 x 17 x 17
    display("Test Successfull!")

test_InceptionC()

'random_test_tensor shape is: ivy.Shape(1, 17, 17, 768)'

'layer 1/10 built'

'layer 2/10 built'

'layer 3/10 built'

'layer 4/10 built'

...

'one 9/20, output shape is:ivy.Shape(1, 17, 17, 192)'

'one 10/20, output shape is:ivy.Shape(1, 17, 17, 768)'

'one 11/20, output shape is:ivy.Shape(1, 17, 17, 192)'

'one 20/20'

'Test Successfull!'

In [ ]:
# note:
# kernal_size in [1,3] list foramt
# and padding in list[list, list] foramt ex: [[1,1],[3,3]] if both dims are unequal, else just single ex: list [3,3]
class InceptionB(ivy.Module):
    def __init__(self, in_channels: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels

    def _build(self, *args, **kwargs):
        self.branch3x3 = self.conv_block(self.in_channels, 384, kernel_size=[3,3], stride=2)
        display(f"layer 1/4 built")

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 64, kernel_size=[1,1])
        display(f"layer 2/4 built")
        self.branch3x3dbl_2 = self.conv_block(64, 96, kernel_size=[3,3], padding=[[1,1],[1,1]])
        display(f"layer 3/4 built")
        self.branch3x3dbl_3 = self.conv_block(96, 96, kernel_size=[3,3], stride=2)
        display(f"layer 4/4 built")

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        branch3x3 = self.branch3x3(x)
        display(f"one 1/20, output shape is:{branch3x3.shape}")

        branch3x3dbl = self.branch3x3dbl_1(x)
        display(f"one 2/20, output shape is:{branch3x3dbl.shape}")
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        display(f"one 3/20, output shape is:{branch3x3dbl.shape}")
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        display(f"one 4/20, output shape is:{branch3x3dbl.shape}")

        branch_pool = ivy.max_pool2d(x, [3,3], 2, 0)
        display(f"one 20/20, output shape is:{branch_pool.shape}")

        outputs = [branch3x3, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        display(f"one 20/20")

        return outputs


def test_InceptionB():
    random_test_tensor = ivy.random_normal(shape=(1, 35, 35, 288))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    # N x 288 x 35 x 35
    block = InceptionB(288)
    block(random_test_tensor)
    # N x 768 x 17 x 17
    display("Test Successfull!")

test_InceptionB()

'random_test_tensor shape is: ivy.Shape(1, 35, 35, 288)'

'layer 1/4 built'

'layer 2/4 built'

'layer 3/4 built'

'layer 4/4 built'

...

'one 3/20, output shape is:ivy.Shape(1, 35, 35, 96)'

'one 4/20, output shape is:ivy.Shape(1, 17, 17, 96)'

'one 20/20, output shape is:ivy.Shape(1, 17, 17, 288)'

'one 20/20'

'Test Successfull!'

In [ ]:
# note:
# kernal_size in [1,3] list foramt
# and padding in list[list, list] foramt ex: [[1,1],[3,3]] if both dims are unequal, else just single ex: list [3,3]
class InceptionA(ivy.Module):
    def __init__(self, in_channels: int, pool_features: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels
        self.pool_features = pool_features

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 64, kernel_size=[1,1])
        display(f"layer 1/7 built")

        self.branch5x5_1 = self.conv_block(self.in_channels, 48, kernel_size=[1,1])
        display(f"layer 2/7 built")
        self.branch5x5_2 = self.conv_block(48, 64, kernel_size=[5,5], padding=[[2,2],[2,2]])
        display(f"layer 3/7 built")

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 64, kernel_size=[1,1])
        display(f"layer 4/7 built")
        self.branch3x3dbl_2 = self.conv_block(64, 96, kernel_size=[3,3], padding=[[1,1],[1,1]])
        display(f"layer 5/7 built")
        self.branch3x3dbl_3 = self.conv_block(96, 96, kernel_size=[3,3], padding=[[1,1],[1,1]])
        display(f"layer 6/7 built")

        self.branch_pool = self.conv_block(self.in_channels, self.pool_features, kernel_size=[1,1])
        display(f"layer 7/7 built")

#         self.avg_pool = ivy.AvgPool2D(3, 1, 1)

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        branch1x1 = self.branch1x1(x)
        display(f"one 1/20")

        branch5x5 = self.branch5x5_1(x)
        display(f"one 2/20")
        branch5x5 = self.branch5x5_2(branch5x5)
        display(f"one 3/20")

        branch3x3dbl = self.branch3x3dbl_1(x)
        display(f"one 4/20")
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        display(f"one 5/20")
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        display(f"one 6/20")

        branch_pool = ivy.avg_pool2d(x, [3,3], [1,1], [[1,1],[1,1]])
#         branch_pool = self.avg_pool(x)
        display(f"one 7/20")
        branch_pool = self.branch_pool(branch_pool)
        display(f"one 8/20")

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        display(f"one 20/20")

        return outputs


def test_InceptionA():
    random_test_tensor = ivy.random_normal(shape=(1, 35, 35, 192))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    # N x 192 x 35 x 35
    block = InceptionA(192, pool_features=32)
    block(random_test_tensor)
    # N x 256 x 35 x 35
    display("Test Successfull!")

test_InceptionA()

'random_test_tensor shape is: ivy.Shape(1, 35, 35, 192)'

'layer 1/7 built'

'layer 2/7 built'

'layer 3/7 built'

'layer 4/7 built'

...

'one 6/20'

'one 7/20'

'one 8/20'

'one 20/20'

'Test Successfull!'

In [ ]:
# note:
# kernal_size in [1,3] list foramt
# and padding in list[list, list] foramt ex: [[1,1],[3,3]] if both dims are unequal, else just single ex: list [3,3]
class InceptionA(ivy.Module):
    def __init__(self, in_channels: int, pool_features: int, conv_block: Optional[Callable[..., ivy.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            self.conv_block = BasicConv2d
        self.in_channels = in_channels
        self.pool_features = pool_features

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 64, kernel_size=[1,1])
        display(f"layer 1/7 built")

        self.branch5x5_1 = self.conv_block(self.in_channels, 48, kernel_size=[1,1])
        display(f"layer 2/7 built")
        self.branch5x5_2 = self.conv_block(48, 64, kernel_size=[5,5], padding=[[2,2],[2,2]])
        display(f"layer 3/7 built")

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 64, kernel_size=[1,1])
        display(f"layer 4/7 built")
        self.branch3x3dbl_2 = self.conv_block(64, 96, kernel_size=[3,3], padding=[[1,1],[1,1]])
        display(f"layer 5/7 built")
        self.branch3x3dbl_3 = self.conv_block(96, 96, kernel_size=[3,3], padding=[[1,1],[1,1]])
        display(f"layer 6/7 built")

        self.branch_pool = self.conv_block(self.in_channels, self.pool_features, kernel_size=[1,1])
        display(f"layer 7/7 built")

#         self.avg_pool = ivy.AvgPool2D(3,1,1)

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        branch1x1 = self.branch1x1(x)
        display(f"InceptionA | branch1x1 1/20, output shape is: {branch1x1.shape}")

        branch5x5 = self.branch5x5_1(x)
        display(f"InceptionA | one 2/20, output shape is: {branch5x5.shape}")
        branch5x5 = self.branch5x5_2(branch5x5)
        display(f"InceptionA | branch5x5_1 3/20, output shape is: {branch5x5.shape}")

        branch3x3dbl = self.branch3x3dbl_1(x)
        display(f"InceptionA | one 4/20, output shape is: {branch3x3dbl.shape}")
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        display(f"InceptionA | one 5/20, output shape is: {branch3x3dbl.shape}")
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        display(f"InceptionA | branch3x3dbl_1 6/20, output shape is: {branch3x3dbl.shape}")

        branch_pool = ivy.avg_pool2d(x, [3,3], [1,1], [[1,1],[1,1]])
#         branch_pool = self.avg_pool(x)
        display(f"InceptionA | one 7/20, output shape is: {branch_pool.shape}")
        branch_pool = self.branch_pool(branch_pool)
        display(f"InceptionA | branch_pool 8/20, output shape is: {branch_pool.shape}")

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        display(f"InceptionA | outputs 20/20")

        return outputs


def test_InceptionA():
    random_test_tensor = ivy.random_normal(shape=(1, 35, 35, 192))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    # N x 192 x 35 x 35
    block = InceptionA(192, pool_features=32)
    block(random_test_tensor)
    # N x 256 x 35 x 35
    display("Test Successfull!")

test_InceptionA()

'random_test_tensor shape is: ivy.Shape(1, 35, 35, 192)'

'layer 1/7 built'

'layer 2/7 built'

'layer 3/7 built'

'layer 4/7 built'

...

'InceptionA | branch3x3dbl_1 6/20, output shape is: ivy.Shape(1, 35, 35, 96)'

'InceptionA | one 7/20, output shape is: ivy.Shape(1, 35, 35, 192)'

'InceptionA | branch_pool 8/20, output shape is: ivy.Shape(1, 35, 35, 32)'

'InceptionA | outputs 20/20'

'Test Successfull!'

In [ ]:
# note:
# kernal_size in [1,3] list foramt
# and padding in list[list, list] foramt ex: [[1,1],[3,3]] if both dims are unequal, else just single ex: list [3,3]
class InceptionV3(ivy.Module):
    def __init__(self,
        num_classes: int = 1000,
        dropout: float = 0.5,) -> None:
        self.num_classes = num_classes
        self.dropout = dropout
        super().__init__()


    def _build(self, *args, **kwargs):
        # inception_blocks = [BasicConv2d, InceptionA, InceptionB, InceptionC, InceptionD, InceptionE, InceptionAux]
        conv_block = BasicConv2d
        inception_a = InceptionA
        inception_b = InceptionB
        inception_c = InceptionC
        inception_d = InceptionD
        inception_e = InceptionE
        inception_aux = InceptionAux

        self.Conv2d_1a_3x3 = conv_block(3, 32, kernel_size=[3,3], stride=2)
        display(f"layer 1/22 built")
        self.Conv2d_2a_3x3 = conv_block(32, 32, kernel_size=[3,3])
        display(f"layer 2/22 built")
        self.Conv2d_2b_3x3 = conv_block(32, 64, kernel_size=[3,3], padding=[[1,1],[1,1]])
        display(f"layer 3/22 built")
        self.maxpool1 = ivy.MaxPool2D([3,3], 2, 0)
        display(f"layer 4/22 built")
        self.Conv2d_3b_1x1 = conv_block(64, 80, kernel_size=[1,1])
        display(f"layer 5/22 built")
        self.Conv2d_4a_3x3 = conv_block(80, 192, kernel_size=[3,3])
        display(f"layer 6/22 built")
        self.maxpool2 = ivy.MaxPool2D([3,3], 2, 0)
        display(f"layer 7/22 built")
        self.Mixed_5b = inception_a(192, pool_features=32)
        display(f"layer 8/22 built")
        self.Mixed_5c = inception_a(256, pool_features=64)
        display(f"layer 9/22 built")
        self.Mixed_5d = inception_a(288, pool_features=64)
        display(f"layer 10/22 built")
        self.Mixed_6a = inception_b(288)
        display(f"layer 11/22 built")
        self.Mixed_6b = inception_c(768, channels_7x7=128)
        display(f"layer 12/22 built")
        self.Mixed_6c = inception_c(768, channels_7x7=160)
        display(f"layer 13/22 built")
        self.Mixed_6d = inception_c(768, channels_7x7=160)
        display(f"layer 14/22 built")
        self.Mixed_6e = inception_c(768, channels_7x7=192)
        display(f"layer 15/22 built")

        # if is used only when the model is in training mode
#         self.AuxLogits = inception_aux(768, num_classes)
#         display(f"layer 16/22 built")

        self.Mixed_7a = inception_d(768)
        display(f"layer 17/22 built")
        self.Mixed_7b = inception_e(1280)
        display(f"layer 18/22 built")
        self.Mixed_7c = inception_e(2048)
        display(f"layer 19/22 built")
        self.avgpool = ivy.AdaptiveAvgPool2d((1, 1))
        display(f"layer 20/22 built")
        self.dropout = ivy.Dropout(prob=self.dropout)
        display(f"layer 21/22 built")
        self.fc = ivy.Linear(2048, self.num_classes)
        display(f"layer 22/22 built")


    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        display(f"input shape is:{x.shape}")

        # N x 3 x 299 x 299
        x = self.Conv2d_1a_3x3(x)
        # N x 32 x 149 x 149
        display(f"v3 1/23, output shape is:{x.shape}")

        x = self.Conv2d_2a_3x3(x)
        # N x 32 x 147 x 147
        display(f"v3 2/23, output shape is:{x.shape}")

        x = self.Conv2d_2b_3x3(x)
        # N x 64 x 147 x 147
        display(f"v3 3/23, output shape is:{x.shape}")

        x = self.maxpool1(x)
        # N x 64 x 73 x 73
        display(f"v3 4/23, output shape is:{x.shape}")

        x = self.Conv2d_3b_1x1(x)
        # N x 80 x 73 x 73
        display(f"v3 5/23, output shape is:{x.shape}")

        x = self.Conv2d_4a_3x3(x)
        # N x 192 x 71 x 71
        display(f"v3 6/23, output shape is:{x.shape}")

        x = self.maxpool2(x)
        # N x 192 x 35 x 35
        display(f"v3 7/23, output shape is:{x.shape}")

        x = self.Mixed_5b(x)
        # N x 256 x 35 x 35
#         display(f"v3 8/23, output type is:{type(x)}")
#         display(f"v3 8/23, output len is:{len(x)}")
        display(f"v3 8/23, output is:{x.shape}")


        x = self.Mixed_5c(x)
        # N x 288 x 35 x 35
        display(f"v3 9/23, output shape is:{x.shape}")

        x = self.Mixed_5d(x)
        # N x 288 x 35 x 35
        display(f"v3 10/23, output shape is:{x.shape}")

        x = self.Mixed_6a(x)
        # N x 768 x 17 x 17
        display(f"v3 11/23, output shape is:{x.shape}")

        x = self.Mixed_6b(x)
        # N x 768 x 17 x 17
        display(f"v3 12/23, output shape is:{x.shape}")

        x = self.Mixed_6c(x)
        # N x 768 x 17 x 17
        display(f"v3 13/23, output shape is:{x.shape}")

        x = self.Mixed_6d(x)
        # N x 768 x 17 x 17
        display(f"v3 14/23, output shape is:{x.shape}")

        x = self.Mixed_6e(x)
        # N x 768 x 17 x 17
        display(f"v3 15/23, output shape is:{x.shape}")

#         aux = self.AuxLogits(x)
#         # N x 768 x 17 x 17
#         display(f"v3 16/23, output shape is:{x.shape}")

        x = self.Mixed_7a(x)
        # N x 1280 x 8 x 8
        display(f"v3 17/23, output shape is:{x.shape}")

        x = self.Mixed_7b(x)
        # N x 2048 x 8 x 8
        display(f"v3 18/23, output shape is:{x.shape}")

        x = self.Mixed_7c(x)
        # N x 2048 x 8 x 8
        display(f"v3 19/23, output shape is:{x.shape}")

        # Adaptive average pooling
        display(f"Inception3 | input shape to adaptive_avg_pool2d is:{x.shape}")
        x = ivy.permute_dims(x, (0, 3, 1, 2))
        display(f"Inception3 | permuted input shape to adaptive_avg_pool2d is:{x.shape}")
        x = self.avgpool(x)
        display(f"Inception3 | output shape from adaptive_avg_pool2d is:{x.shape}")
        x = ivy.permute_dims(x, (0, 2, 3, 1))
        display(f"Inception3 | permuted output shape from adaptive_avg_pool2d is:{x.shape}")

        # N x 2048 x 1 x 1
        display(f"v3 20/23, avgpool output shape is:{x.shape}")

        x = self.dropout(x)
        # N x 2048 x 1 x 1
        display(f"v3 21/23, dropout output shape is:{x.shape}")

        x = ivy.flatten(x, start_dim=1)
        # N x 2048
        display(f"v3 22/23, flatten output shape is:{x.shape}")

        x = self.fc(x)
        # N x 1000 (num_classes)
        display(f"fc done 23/23")

#         return x, aux
        return x

def test_InceptionV3():
    ivy.set_backend('torch')
    random_test_tensor = ivy.random_normal(shape=(1, 256, 256, 3))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")

    block = InceptionV3()
    block(random_test_tensor)
    display("Test Successfull!")

test_InceptionV3()

'random_test_tensor shape is: ivy.Shape(1, 256, 256, 3)'

'layer 1/22 built'

'layer 2/22 built'

'layer 3/22 built'

'layer 4/22 built'

...

'3/20, branch3x3 output shape is:ivy.Shape(1, 6, 6, 768)'

'4/20, branch3x3dbl output shape is:ivy.Shape(1, 6, 6, 448)'

'5/20, branch3x3dbl output shape is:ivy.Shape(1, 6, 6, 384)'

'6/20, branch3x3dbl output shape is:ivy.Shape(1, 6, 6, 768)'

RuntimeError: ignored